In [37]:
import pandas as pd
import numpy as np
import math
from sklearn.naive_bayes import GaussianNB
from sklearn import metrics

#Read from the files.

impr = pd.read_csv('imp.20131019.txt', sep = '\t', header = None)
clk =  pd.read_csv('clk.20131019.txt', sep = '\t', header = None)



In [38]:
#Delete the log type field from both the data frames
del impr[2]
del clk[2]

# Converting the dataframes to numpy arrays
impr_np = impr.values
click = clk.values

b = []
## Concatenating the BidId and Timestamp to get unique values for each row.
for i in range(len(impr_np)):
    b.append(impr_np[i][0]+ str(impr_np[i][1]))
    
col = np.array(b).reshape(len(impr_np),1)

#Deleting the BidId and timestamp fields from the data and appending the concatenated values.
n_impr = np.delete(impr_np,0,1)
ne_impr = np.delete(n_impr,0,1)

new_impr = np.append(ne_impr,col,axis=1)

#Repeating the above steps for the clicks data
c= []

for i in range(len(click)):
    c.append(click[i][0]+ str(click[i][1]))

colc = np.array(c).reshape(len(click),1)

n_click = np.delete(click,0,1)
ne_click = np.delete(n_click,0,1)
new_click =np.append(ne_click,colc,axis=1)


In [39]:
#Appending the log type to the impressions data
ones_ar = np.ones((len(new_impr),1), dtype=int)
print(ones_ar.shape)

new_impr = np.append(new_impr,ones_ar,axis=1)

# Changing the log type to 2 in the impression matrix based on the clicks data
for i in range(len(new_impr)):
    if new_impr[i][21] in new_click[:,21]:
        new_impr[i][22] = 2


(228133, 1)


In [40]:
print(new_impr.shape)

(228133, 23)


In [41]:
# Removing the unneccessary features
remove_indices = [0,1,2,3,7,8,13,16,18,19,20,21]

for i in range(len(remove_indices)):
    new_impr = np.delete(new_impr,remove_indices[i]-i,1)
print(new_impr.shape)
print(new_impr[0])

# City, adexchange, domain,adslotid,width, height,view,floorprice,creative id, paying price


(228133, 11)
[234 2 '33235ca84c5fee9254e6512a41b3ad5e' '3061584349' 728 90 'OtherView' 5
 7330 48 1]


In [42]:
#Changing the values in the dataset(hashed,ID's) to numeric values
for col_num in range(10):
    if col_num != 1:
        uni_npr = np.unique(new_impr[:,col_num])
        uni_list = uni_npr.tolist()
        for row_num in range(len(new_impr)):
            new_impr[row_num][col_num] = uni_list.index(new_impr[row_num][col_num])
print(new_impr.shape)
print(new_impr[0])

#Data pre-processing done.

(228133, 11)
[18 2 927 5566 10 1 4 2 14 48 1]


In [80]:
#Dividing the data into training data and lables

X = new_impr[:,:10]
Y = new_impr[:,10]

Y=Y.astype('int')
for i in range(len(X)):
    for j in range(len(X[i])):
        X[i][j] = float(X[i][j])

for i in range(len(Y)):
    Y[i] = float(Y[i])

x_len = len(X)

#Splitting the data - Training(75%), Testing(25%)
split_percent = 0.75

len_train = math.floor(x_len * split_percent);

X_train = X[:len_train]
Y_train = Y[:len_train]


In [81]:
#Gusssian Naive Bayes Classifier

clf = GaussianNB()
clf = clf.fit(X_train,Y_train)

#Validation dataset for testing

X_validation = X[len_train:len(X)]
Y_validation = Y[len_train:len(Y)]

#Prediction
output_Predicted = clf.predict(X_validation);

In [82]:
#Measuring accuracy using, recall, precision
accuracy_training = metrics.accuracy_score(output_Predicted,Y_validation)
print("Accuracy on the Training Data set is :")
print(accuracy_training* 100)

recall_training = metrics.recall_score(output_Predicted,Y_validation)
print("Recall on the Training Data set is :")
print(recall_training* 100)


precision_training = metrics.precision_score(output_Predicted,Y_validation)
print("Precision on the Training Data set is :")
print(precision_training* 100)


Accuracy on the Training Data set is :
99.9614265175
Recall on the Training Data set is :
99.9614265175
Precision on the Training Data set is :
100.0


In [83]:
output_Prob = clf.predict_proba(X_validation);

acc_list = []

for i in range(len(Y_validation)):
    if(Y_validation[i] == 2):
        acc_list.append(i)
print(acc_list)
print(output_Prob.shape)

[1640, 1940, 5964, 8663, 14560, 15332, 20748, 20781, 23816, 25270, 31312, 36322, 38345, 39294, 40056, 42359, 42769, 50117, 52353, 52938, 53312, 54443]
(57034, 2)


In [84]:
ac_ctr = len(click)/len(impr_np)
ctr = 0;

for i in range(len(acc_list)):
    ctr = ctr + output_Prob[acc_list[0]][1]

#for i in range(len(output_Prob)):
#    ctr = ctr + output_Prob[i][1]

ctr = ctr/len(acc_list)
print("predicted ctr ", ctr)
print("actual ctr ",ac_ctr)

predicted ctr  0.00570967548537
actual ctr  0.0003638228577189622


In [85]:
clf1 = GaussianNB()
clf1.fit(X,Y)

tot_pred_ctr =0

op_pred_prob = clf.predict_proba(X)

for i in range(len(op_pred_prob)):
    tot_pred_ctr = tot_pred_ctr + op_pred_prob[i][1]
tot_pred_ctr = tot_pred_ctr/len(op_pred_prob)

print("total predicted probability ctr= ", tot_pred_ctr)

total predicted probability ctr=  0.00156868126037
